## PDF SALES DATA EXTRACTION

<h5>I used <span style="color:blue">PyPDF4</span> and <span style="color:blue">regex</span> library for extract text from pdf.</h5>

In [58]:
import PyPDF4
import re
import pandas as pd

### Collection of all data fetching methods.

<p>Data_fetched_method class contains several methods for extracting specific information from a PDF document.</p>

In [61]:
class Data_fetched_method:
    
    def get_invoiceNumber(pdf_text):
        in_match = re.search(r'Invoice\s*No.\s*(\S+)', pdf_text)
        if in_match:
            invoice_number = in_match.group(1)
            return invoice_number
            # print("Invoice number: "+invoice_number)
            
        else:
            print("Invoice number not found in the text.")
            
    def get_invoiceDate(pdf_text):
        id_match = re.search(r'Invoice\s*Date\s*(\S+)', pdf_text)
        if id_match:
            invoice_date = id_match.group(1)
            return invoice_date
            # print("Invoice date: "+invoice_date)
        else:
            print("Invoice date not found in the text.")
            
    def get_orderDate(pdf_text):
        od_match = re.search(r'Order\s*Date\s*(\S+)', pdf_text)
        if od_match:
            order_date = od_match.group(1)
            return order_date
            # print("Order date: "+order_date)
        else:
            print("Order date not found in the text.")
            
    def get_productDetails(pdf_text):
        pattern = r"Order No\.\s+(.*?)\s+Fold Here"
        pd_match = re.search(pattern, pdf_text, re.DOTALL)
        if pd_match:
            result2 = pd_match.group(1)
            lines2 = result2.split("\n")
            SKU = lines2[0]
            SIZE = lines2[1]
            QTY = lines2[2]
            COLOR = lines2[3]
            ORDER_NO = lines2[4]

            product_details = [SKU, SIZE, QTY, COLOR, ORDER_NO]
            return product_details
        
            # print("SKU = "+SKU)
            # print("Size = "+SIZE)
            # print("Qty = "+QTY)
            # print("Color = "+COLOR)
            # print("Order No. = "+ORDER_NO)
        else:
            print("No match found in product details.")
        
    def get_other_details(pdf_text):
        other_details_result = re.search(r'Total\n(.*?)\nOther Charges', pdf_text, re.DOTALL)
        if other_details_result:
            extracted_text = other_details_result.group(1).strip()
            lines = extracted_text.split("\n")
            lines1 = extracted_text.split('\n')[:len(lines)-9]
            
            # other details
            description = ' '.join(filter(None, lines1))
            hsn = lines[-9]
            qty = lines[-8]
            gross_amount = lines[-7]
            discount = lines[-6]
            taxable_value = lines[-5]

            # tax
            taxes_string = lines[-4]
            match1 = re.search(r'@(.+?)%', taxes_string)
            if match1:
                taxes = int(float(match1.group(1)))

            # total
            total = lines[-1]
            total = total[3:len(total)]

            other_details = {"description":description, "hsn":hsn, "qty":qty, "gross amount":gross_amount, "discount":discount, "taxable value":taxable_value, "taxes":taxes, "total":total}
            return other_details
            
            # print("Description = "+description)
            # print("HSN = "+hsn)
            # print("Qty = "+qty)
            # print("Gross Amount = "+gross_amount[3:len(gross_amount)])
            # print("Discount = "+discount[3:len(discount)])
            # print("Taxable Value = "+taxable_value[3:len(taxable_value)])
            # print("Taxes = "+str(taxes))
            # print("Total = "+total)

### Processing a PDF document

In [62]:
pdf_file = open('all_sub_order.pdf', 'rb')
pdf_reader = PyPDF4.PdfFileReader(pdf_file)

<h3> Utilizes methods from the <span style="color:blue">Data_fetched_method</span> class </h3>
<p>pdf_model is actually a generator function which yield the details one by one.</p>

In [63]:
def pdf_model(pdf_reader):
    for page_num in range(1, pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        pdf_text = page.extractText()
        # print(pdf_text)
        
        # get invoice number
        invoiceNumber = Data_fetched_method.get_invoiceNumber(pdf_text)
        
        # get invoice date
        invoiceDate = Data_fetched_method.get_invoiceDate(pdf_text)
        
        # get order date
        orderDate = Data_fetched_method.get_orderDate(pdf_text)
        
        # get product details
        productDetails_list = Data_fetched_method.get_productDetails(pdf_text)
        
        
        # get other details
        otherDetails_list = Data_fetched_method.get_other_details(pdf_text)
        od = []
        
        details = {"Invoice number": invoiceNumber, 
                   "Invoice date":invoiceDate, 
                   "Order date":orderDate, 
                   "Sku":productDetails_list[0],
                   "Size":productDetails_list[1],
                   "Qty":productDetails_list[2],
                   "Color": productDetails_list[3],
                   "Order No.": productDetails_list[4],
                   "Description":otherDetails_list["description"],
                   "Hsn":otherDetails_list["hsn"],
                   "Qty":otherDetails_list["qty"],
                   "Gross amount":otherDetails_list["gross amount"],
                   "Discount":otherDetails_list["discount"],
                   "Taxable Value":otherDetails_list["taxable value"],
                   "Taxes":otherDetails_list["taxes"],
                   "Total":otherDetails_list["total"],
                   }
        yield details
        
        # print(invoiceNumber)
        # print(invoiceDate)
        # print(orderDate)
        # print(productDetails_list)
        # print(otherDetails_list['description'])

In [64]:
obj = pdf_model(pdf_reader)
df = pd.DataFrame(obj)

In [65]:
df.head(2)

,Invoice number,Invoice date,Order date,Sku,Size,Qty,Color,Order No.,Description,Hsn,Gross amount,Discount,Taxable Value,Taxes,Total
0,rv23a235578,20.03.2023,18.03.2023,1443290244,M,1,Mustard,321189690290_1,Shivansh Creation Women Reyon Printed Matern...,6204,Rs.404.00,Rs.0.00,Rs.384.76,5,404.00
1,rv23a235583,20.03.2023,19.03.2023,1838955934,Un Stitched,1,NA,370533673879_1,Shivansh Creation Women's Crepe Printed Unst...,540710,Rs.334.00,Rs.0.00,Rs.318.10,5,334.00


### Feature Engineering

In [66]:
column_names = df.columns.tolist()
column_datatype = df.dtypes.tolist()

<h5>Remove "Rs." on column number 11,12,13 </h5>

In [67]:
for i, (column, datatype) in enumerate(zip(column_names, column_datatype)):
    print(i+1, "-->", column, "-->", datatype)

1 --> Invoice number --> object
2 --> Invoice date --> object
3 --> Order date --> object
4 --> Sku --> object
5 --> Size --> object
6 --> Qty --> object
7 --> Color --> object
8 --> Order No. --> object
9 --> Description --> object
10 --> Hsn --> object
11 --> Gross amount --> object
12 --> Discount --> object
13 --> Taxable Value --> object
14 --> Taxes --> int64
15 --> Total --> object


In [68]:
df["Gross amount"] = df["Gross amount"].str.replace("Rs.", "")
df["Discount"] = df["Discount"].str.replace("Rs.", "")
df["Taxable Value"] = df["Taxable Value"].str.replace("Rs.", "")

In [69]:
df

,Invoice number,Invoice date,Order date,Sku,Size,Qty,Color,Order No.,Description,Hsn,Gross amount,Discount,Taxable Value,Taxes,Total
0,rv23a235578,20.03.2023,18.03.2023,1443290244,M,1,Mustard,321189690290_1,Shivansh Creation Women Reyon Printed Matern...,6204,404.00,0.00,384.76,5,404.00
1,rv23a235583,20.03.2023,19.03.2023,1838955934,Un Stitched,1,NA,370533673879_1,Shivansh Creation Women's Crepe Printed Unst...,540710,334.00,0.00,318.10,5,334.00
2,rv23a235579,20.03.2023,19.03.2023,1838955934,Un Stitched,1,NA,540109806053_1,Shivansh Creation Women's Crepe Printed Unst...,540710,334.00,0.00,318.10,5,334.00
3,rv23a235576,20.03.2023,18.03.2023,2146626382,Un Stitched,1,NA,435621494823_1,Shivansh Creation Women's Crepe Printed Unst...,540710,334.00,0.00,318.10,5,334.00
4,rv23a235589,20.03.2023,19.03.2023,44858665,Un Stitched,1,NA,973644401525_1,Shivansh Creation Women's Crepe Printed Unst...,540710,334.00,0.00,318.10,5,334.00
5,rv23a235580,20.03.2023,18.03.2023,MK_BlackTop,S,1,Black,154238439003_1,Shivansh Creation Women's My Swag Solid A-li...,610442,258.00,0.00,245.71,5,258.00
6,rv23a235584,20.03.2023,18.03.2023,MK_BlackTop,M,1,Black,380584361580_1,Shivansh Creation Women's My Swag Solid A-li...,610442,260.00,0.00,247.62,5,260.00
7,rv23a235573,20.03.2023,19.03.2023,MK_BlackTop,M,1,Black,482142932329_1,Shivansh Creation Women's My Swag Solid A-li...,610442,258.00,0.00,245.71,5,258.00
8,rv23a235577,20.03.2023,18.03.2023,MK_PINKTOP,S,1,Pink,255133493368_1,Shivansh Creation Women's My Swag Solid A-li...,610442,272.00,0.00,259.05,5,272.00
9,rv23a235574,20.03.2023,18.03.2023,MK_PINKTOP,S,1,Pink,291450434388_1,Shivansh Creation Women's My Swag Solid A-li...,610442,258.00,0.00,245.71,5,258.00
